In [494]:
import pandas as pd
import datetime
import plotly_express as px
import numpy as np
df = pd.read_pickle('health2.pickle')

In [ ]:
df.columns

In [ ]:
df.index = pd.to_datetime(df.index)
df.sort_index(inplace=True)
df

In [ ]:
# Add column for average speed on running
TRIP_DISTANCE_METERS = 6000
MARGIN_PERCANTAGE = 8
diff_meters = (MARGIN_PERCANTAGE / 100) * TRIP_DISTANCE_METERS
# TODO: only calc for trip distance
df['avg_speed_running_trip'] = df.apply(lambda row: (row.running_duration_seconds/60) / row.running_distance_meters if row.running_distance_meters and row.running_distance_meters - diff_meters <= row.running_distance_meters <= row.running_distance_meters + diff_meters and row.running_distance_meters >= () else np.NaN, axis=1)


In [ ]:
# combining where date occurs twice
relevant_columns = ('date', 'bodymass_kg_mean', 'stepcount_count_sum', 'distancewalkingrunning_km_sum', 'bodymass_kg', 'bloodpressuresystolic_mmHg', 'bloodpressurediastolic_mmHg', 'walking_distance_meters', 'running_distance_meters', 'avg_speed_running_trip')
strategy = {_: 'first' for _ in relevant_columns}
df = df.groupby(df.index).agg(strategy)
df.sort_index()
df.drop('date', axis=1, inplace=True)
df.tail(40)

In [ ]:
df['running_distance_garmin'] = df.running_distance_meters.apply(lambda x: x / 1000 if not pd.isna(x) else 0)
df['walking_distance_garmin'] = df.walking_distance_meters.apply(lambda x: x / 1000 if not pd.isna(x) else 0)
df['walking_running_distance_applehealth'] = df.distancewalkingrunning_km_sum.apply(lambda x: x if not pd.isna(x) else 0)

df['walking_km'] = df.apply(lambda row: row.walking_running_distance_applehealth - row.running_distance_garmin if row.walking_running_distance_applehealth >= row.walking_distance_garmin + row.running_distance_garmin else row.walking_running_distance_applehealth, axis=1)
df['running_km'] = df.apply(lambda row: row.running_distance_garmin, axis=1)

df

In [ ]:
df['total_distance_km'] = df.apply(lambda row: row.running_km + row.walking_km, axis=1)

In [ ]:
# Fixing blood preassure
for _ in ('bloodpressurediastolic_mmHg', 'bloodpressuresystolic_mmHg', 'bodymass_kg'):
    df[_].replace(0.0, np.NaN, inplace=True)
    df[_].interpolate(method='linear', limit_direction='backward', inplace=True)

# filtering
df_all = df.copy()
f, t = datetime.date(2020, 1, 1), datetime.date(2023, 1, 1)
dff = df.query('date > @f & date < @t')
df_filtered = dff.copy()

In [ ]:
dff = dff.resample('W').sum()
dff['walking_running_km_mean'] = dff.total_distance_km.mean()

dff

In [ ]:
b = px.bar(dff, y=['walking_km', 'running_km'], title="Weekly distance", height=500)
b.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01), legend_title_text=None)
l = px.line(dff, y="walking_running_km_mean")
b.add_traces(l.data)

In [ ]:
dff = df_filtered.resample('W').max()
both = ["bloodpressuresystolic_mmHg", "bloodpressurediastolic_mmHg"]
for _ in both:
    dff[_] = dff[_].rolling(3).mean()
l = px.line(dff, y=both, title="Weekly blood pressure", height=500)
l.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01), legend_title_text=None)

In [ ]:
dff = df_filtered.resample('W').max()
dff['bodymass_kg'] = dff['bodymass_kg'].rolling(5).mean()
l = px.line(dff, y='bodymass_kg', title="Average weight", height=500)
l.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01), legend_title_text=None)